This notebook will create 1 dataframe from all four modules
Shuffle and Randomly partition that one dataframe 

80%train + Cross validation: This data set is used to compare the performances of the prediction algorithms that were created based on the training set. We choose the algorithm that has the best performance.

##############
20%test: Once we have chosen our preferred prediction algorithm but we don't know yet how it's going to perform on 
completely new real-world data. So, we apply our chosen prediction algorithm on our test set in order to see how 
it's going to perform so we can have an idea about our algorithm's performance on new data.
##############

In [1]:
def generateDF(featuresFile, profiledFile):
    #reads featurefiles and projects 3 columns (ppn, sizeGB, Ytime)
    #reads profiledFile (the table from profiling) and reads all 55 columns
    #generates one dataframe with 58 columns and n rows (=size of profiled data from csv)
    import pandas as pd
    from IPython.display import display

    header='ft1 ft2 ft3 ft4 ft5 ft6 ft7 ft8 ft9 ft10 ft11 ft12 ft13 ft14 ft15 ft16 ft17 \
    ft18 ft24 ft25 ft19 ft39 ft20 ft33 ft21 ft35 ft22 ft23 ft34 ft36 ft37 ft38 ft40 ft41 \
    ft42 ft43 ft44 ft45 ft46 ft48 ft47 ft49 ft51 ft50 ft52 ft53 ft54 ft55 ft26 ft27 ft28 ft29 ft30 ft31 ft32'
    
    def value(item):
        return item[item.find('=')+1:]
    
    print("Reading features from %s" % featuresFile)
    df_features = pd.read_table(featuresFile, header=None, delimiter=',',
                       converters={i:value for i in range(55)},
                       names=header.split())
    df_features = df_features.astype(float)
    #print(df_features.head(2))
    
    ################################
    #read profiledFile
    df_profiled = pd.read_csv(profiledFile)
    if 'Unnamed: 0' in df_profiled.columns:
        del df_profiled['Unnamed: 0']
    df_profiled = df_profiled.astype(float)
    rows = df_profiled.shape[0]
    
    #project columns of use
    df_profiled = df_profiled[['ppn','sizeGB','Y_time']]
    print("Reading profiled file %s\t: %s" % (profiledFile,str(df_profiled.shape)))
    
    ################################
    # create dataframe from featuresFile static program features with same #ROWs
    frames = [df_features for i in range(rows)]
    program_ft = pd.concat(frames)
    program_ft.reset_index(inplace=True)
    del program_ft['index']
    print("Shape of program static features\t\t\t\t\t: %s " % str(program_ft.shape))
    
    ################################
    # concatenate static features (55) with profiled data (3)
    ft_plus_profiled = pd.concat([df_profiled, program_ft], axis=1, join_axes=[df_profiled.index])
    ft_plus_profiled['y_time']=ft_plus_profiled['Y_time'] # so that y_time is last column
    del ft_plus_profiled['Y_time']
    print("Returning concatenated data frame (ft + profiled data)\t\t\t: %s" % str(ft_plus_profiled.shape))
    #display(ft_plus_profiled.tail(5))    
    
    return ft_plus_profiled

In [2]:
# stack two dataframes on top of each other
def stackDFs(df1, df2):
    import copy
    import pandas as pd
    
    frames = [copy.deepcopy(df1), copy.deepcopy(df2)]
    both = pd.concat(frames)
    both.reset_index(inplace=True)
    del both['index']
    return both

In [3]:
#
############################################################################################
############################################################################################
################################## cdhitdup ################################################
############################################################################################
############################################################################################
#

In [4]:
df_cdhitdup = generateDF('./features/cdhitdup_singlecmd_features.txt','./profiledcsvfiles/cdhitdup_ppn_sizeGB_Ytime.csv')
df_cdhitdup.shape

Reading features from ./features/cdhitdup_singlecmd_features.txt
Reading profiled file ./profiledcsvfiles/cdhitdup_ppn_sizeGB_Ytime.csv	: (32, 3)
Shape of program static features					: (32, 55) 
Returning concatenated data frame (ft + profiled data)			: (32, 58)


(32, 58)

In [5]:
from IPython.display import display

In [6]:
#
############################################################################################
############################################################################################
##################################### frhit ################################################
############################################################################################
############################################################################################
#

In [7]:
df_frhit = generateDF('./features/frhit_singlecmd_features.txt', './profiledcsvfiles/frhit_ppn_sizeGB_Ytime.csv')

Reading features from ./features/frhit_singlecmd_features.txt
Reading profiled file ./profiledcsvfiles/frhit_ppn_sizeGB_Ytime.csv	: (96, 3)
Shape of program static features					: (96, 55) 
Returning concatenated data frame (ft + profiled data)			: (96, 58)


In [8]:
#
############################################################################################
############################################################################################
##################################### velvetH ##############################################
############################################################################################
############################################################################################
#

In [9]:
df_hvelvetH = generateDF('./features/velveth_singlecmd_features.txt','./profiledcsvfiles/velvetH_ppn_sizeGB_Ytime.csv')

Reading features from ./features/velveth_singlecmd_features.txt
Reading profiled file ./profiledcsvfiles/velvetH_ppn_sizeGB_Ytime.csv	: (97, 3)
Shape of program static features					: (97, 55) 
Returning concatenated data frame (ft + profiled data)			: (97, 58)


In [10]:
#
############################################################################################
############################################################################################
##################################### velvetG ##############################################
############################################################################################
############################################################################################
#

In [11]:
df_gvelvetG = generateDF('./features/velvetg_singlecmd_features.txt','./profiledcsvfiles/velvetG_ppn_sizeGB_Ytime.csv')

Reading features from ./features/velvetg_singlecmd_features.txt
Reading profiled file ./profiledcsvfiles/velvetG_ppn_sizeGB_Ytime.csv	: (97, 3)
Shape of program static features					: (97, 55) 
Returning concatenated data frame (ft + profiled data)			: (97, 58)


### Stack 4 dfs: df_gvelvetG, df_hvelvetH, df_cdhitdup, df_frhit

In [12]:
import numpy as np
import pandas as pd

In [13]:
frames = [df_cdhitdup, df_frhit, df_hvelvetH, df_gvelvetG]
result = pd.concat(frames)
result.reset_index(inplace=True)
del result['index']

In [14]:
features = result.columns[0:57]
target = [result.columns[57]]

In [15]:
len(features), features

(57,
 Index(['ppn', 'sizeGB', 'ft1', 'ft2', 'ft3', 'ft4', 'ft5', 'ft6', 'ft7', 'ft8',
        'ft9', 'ft10', 'ft11', 'ft12', 'ft13', 'ft14', 'ft15', 'ft16', 'ft17',
        'ft18', 'ft24', 'ft25', 'ft19', 'ft39', 'ft20', 'ft33', 'ft21', 'ft35',
        'ft22', 'ft23', 'ft34', 'ft36', 'ft37', 'ft38', 'ft40', 'ft41', 'ft42',
        'ft43', 'ft44', 'ft45', 'ft46', 'ft48', 'ft47', 'ft49', 'ft51', 'ft50',
        'ft52', 'ft53', 'ft54', 'ft55', 'ft26', 'ft27', 'ft28', 'ft29', 'ft30',
        'ft31', 'ft32'],
       dtype='object'))

In [16]:
len(target), target

(1, ['y_time'])

In [17]:
len(result.columns)

58

In [18]:
###@@@@@@@@@@@@@@@@@@@@@@@@

## Split into Test and Train

In [19]:
from sklearn.model_selection import train_test_split

#copy from concatenated dataframe
ft0 =result.dropna().copy(deep=True)

#shuffle
####################################
ft1  =ft0.iloc[np.random.permutation(len(ft0))]
ft2  =ft1.reset_index(drop=True)

In [20]:
# Train Set, and |validationSet|
ft_subset, testSet = train_test_split(ft2, test_size = 0.2)
ft = ft_subset.copy(deep=True)
ft.shape, testSet.shape, result.shape

((256, 58), (65, 58), (322, 58))

In [21]:
X_train = ft[features]
y_train = ft[target].values.ravel()

X_test = testSet[features]
y_test = testSet[target].values.ravel()

In [22]:
# Store this partition to all modules use this
d = { 'X_train': X_train, 'y_train': y_train, 'X_test':X_test, 'y_test': y_test }

In [23]:
# Write to file
import pickle

with open('datasetIPPD.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [24]:
# Read Back

with open('datasetIPPD.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [25]:
b.keys(), b['X_train'].shape

(dict_keys(['X_train', 'y_train', 'X_test', 'y_test']), (256, 57))

In [26]:
###@@@@@@@@@@@@@@@@@@@@@@@@

In [27]:
import datetime
print("Data Partitioning Date: %s" % datetime.datetime.now().strftime("%Y-%m-%d %H:%M") )

Data Partitioning Date: 2017-02-09 14:28
